In [1]:
#import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
import re




In [2]:
#import data
spam_df = pd.read_csv("D:\\Datasets\\spam.csv")
spam_df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
#inspact data
spam_df.groupby("Category").describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [4]:
#turning spam/ham into numerical data, creating new column called 'spam'
spam_df['spam'] = spam_df['Category'].apply(lambda x: 1 if x == 'spam'else 0)
#x=spam_df.Message
y=spam_df.spam



In [5]:
ps = PorterStemmer()
def clean_text(text):
    # Remove any unwanted characters
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords and stem the words
    words = [ps.stem(word) for word in text.split() if word not in stopwords.words("english")]
    return " ".join(words)

# Apply the clean_text function to the text column of the DataFrame
spam_df["Message"] = spam_df["Message"].apply(clean_text)


In [11]:
x=spam_df.Message

## MultinomialNB

In [12]:
#find the word count and store data as a matrix
cv = CountVectorizer()
x_count = cv.fit_transform(x) 
print(x_count)

  (0, 3052)	1
  (0, 3721)	1
  (0, 5028)	1
  (0, 2049)	1
  (0, 1224)	1
  (0, 1605)	1
  (0, 3130)	1
  (0, 7095)	1
  (0, 3830)	1
  (0, 1603)	1
  (0, 1839)	1
  (0, 3092)	1
  (0, 1037)	1
  (0, 6918)	1
  (1, 4683)	1
  (1, 3861)	1
  (1, 3688)	1
  (1, 7017)	1
  (1, 4708)	1
  (2, 2885)	1
  (2, 2553)	2
  (2, 7061)	1
  (2, 1936)	1
  (2, 7028)	1
  (2, 2672)	2
  :	:
  (5567, 165)	1
  (5567, 4610)	1
  (5568, 3052)	1
  (5568, 3346)	1
  (5568, 2874)	1
  (5568, 2585)	1
  (5569, 6172)	1
  (5569, 4356)	1
  (5569, 4970)	1
  (5570, 2885)	1
  (5570, 6953)	1
  (5570, 3942)	1
  (5570, 4532)	1
  (5570, 6754)	1
  (5570, 5920)	1
  (5570, 1631)	1
  (5570, 2510)	1
  (5570, 2979)	1
  (5570, 3174)	1
  (5570, 3561)	1
  (5570, 897)	1
  (5570, 1429)	1
  (5571, 4452)	1
  (5571, 6599)	1
  (5571, 5472)	1


In [ ]:
#create train/test split
x_train ,x_test , y_train , y_test = train_test_split(x_count, y,test_size=0.3,random_state=42)

In [ ]:
# #find the word count and store data as a matrix
# cv = CountVectorizer()
# x_train_count = cv.fit_transform(x_train.values) 
# x_train_count

In [ ]:
# x_train_count.toarray()

In [ ]:
#train model 
model = MultinomialNB()
model.fit(x_train,y_train)

In [11]:
#pre_test ham
email_ham = ['hey wanna meet up for the game?']
email_ham_count = cv.transform(email_ham) #we use only the transform func
model.predict(email_ham_count)

array([0], dtype=int64)

In [12]:
#pre-test spam
email_spam = ['reward money click']
email_spam_count = cv.transform(email_spam)
model.predict(email_spam_count)

array([1], dtype=int64)

In [13]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")

Accuracy: 0.986244019138756, Precision: 0.9427312775330396, Recall: 0.9553571428571429, F1-score: 0.9490022172949002


In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrics = confusion_matrix(y_test,y_pred)
confusion_matrics

array([[1435,   13],
       [  10,  214]], dtype=int64)

### using TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_c = TfidfVectorizer()
count_wm = tfidf_c.fit_transform(x)
xtrain_tfidf =count_wm.toarray()

In [22]:
x_train ,x_test , y_train , y_test = train_test_split(count_wm, y,test_size=0.3,random_state=42)

model = MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB()

In [25]:
# # test model

y_pred = model.predict(x_test)
# model.score(x_test,y_test)

In [26]:
confusion_matrics = confusion_matrix(y_test,y_pred)
confusion_matrics

array([[1448,    0],
       [  66,  158]], dtype=int64)

In [27]:

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")

Accuracy: 0.9605263157894737, Precision: 1.0, Recall: 0.7053571428571429, F1-score: 0.8272251308900525


#### 2nd way

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False)
x_tf = tf_transformer.fit_transform(x_count)


In [29]:
x_train ,x_test , y_train , y_test = train_test_split(x_tf, y,test_size=0.3,random_state=42)

model = MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB()

In [32]:
y_pred_tf = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")

Accuracy: 0.9455741626794258, Precision: 1.0, Recall: 0.59375, F1-score: 0.7450980392156863


In [33]:
confusion_matrics = confusion_matrix(y_test,y_pred_tf)
confusion_matrics

array([[1448,    0],
       [  91,  133]], dtype=int64)

In [34]:
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(x_count)
x_train ,x_test , y_train , y_test = train_test_split(X_tfidf, y,test_size=0.3,random_state=42)

In [35]:
model.fit(x_train,y_train)

MultinomialNB()

In [36]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")

Accuracy: 0.9605263157894737, Precision: 1.0, Recall: 0.7053571428571429, F1-score: 0.8272251308900525


## GaussianNB

In [37]:
from sklearn.naive_bayes import GaussianNB

X_freq = x_count.toarray()

In [38]:
x_train, x_test, y_train, y_test = train_test_split(X_freq, y, test_size=0.3,random_state=42)

In [39]:
model_2 = GaussianNB()
model_2.fit(x_train,y_train)

GaussianNB()

In [40]:
y_pred = model_2.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")

Accuracy: 0.9102870813397129, Precision: 0.6081871345029239, Recall: 0.9285714285714286, F1-score: 0.734982332155477


## bernoulli


In [10]:
X_binary = (x_count > 0).astype(int)
X_binary

<5572x8709 sparse matrix of type '<class 'numpy.int32'>'
	with 74098 stored elements in Compressed Sparse Row format>

In [45]:
x_train, x_test, y_train, y_test = train_test_split(X_binary, y, test_size=0.2, random_state=42)


In [46]:
from sklearn.naive_bayes import BernoulliNB

model_3 = BernoulliNB()
model_3.fit(x_train,y_train)

BernoulliNB()

In [47]:
y_pred = model_3.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")

Accuracy: 0.9829596412556054, Precision: 0.9779411764705882, Recall: 0.8926174496644296, F1-score: 0.9333333333333333


In [44]:
#the key random_state = 42 same ens